In [1]:
import requests
import wget
import zipfile
import os
import glob
import pandas as pd

In [11]:
#Data Scrapping
asean = ["BRN","KHM","IDN","LAO","MYS","MMR","PHL","SGP","THA","VNM"]

path =os.getcwd()

for ams in asean:

    url = f"https://downloads.climatetrace.org/v02/country_packages/{ams}.zip"

    latest_driver_zip = wget.download(url,"chromedriver.zip")

    with zipfile.ZipFile(latest_driver_zip,'r') as zip_ref:
        zip_ref.extractall(os.path.join(path,ams))
    os.remove(latest_driver_zip)




In [33]:

df_all_country_level = pd.DataFrame()

for ams in asean:
    paths = os.path.join(path,ams,"DATA")
    sectors = [f for f in os.listdir(paths) if os.path.isdir(os.path.join(paths,f))]

    for sector in sectors:
        file_list = glob.glob(os.path.join(paths,sector) + "/*emissions.csv")

        for file in file_list:
            df = pd.read_csv(file)
            df_all_country_level = pd.concat([df_all_country_level,df])
            

In [34]:
df_all_country_level.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21025 entries, 0 to 39
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   iso3_country              21025 non-null  object 
 1   sector                    21025 non-null  object 
 2   subsector                 21025 non-null  object 
 3   start_time                21025 non-null  object 
 4   end_time                  21025 non-null  object 
 5   gas                       21025 non-null  object 
 6   emissions_quantity        17465 non-null  float64
 7   emissions_quantity_units  21025 non-null  object 
 8   temporal_granularity      15341 non-null  object 
 9   created_date              21025 non-null  object 
 10  modified_date             5413 non-null   object 
dtypes: float64(1), object(10)
memory usage: 1.9+ MB


In [41]:
df_all_country_level["start_time"] = pd.to_datetime(df_all_country_level["start_time"])
df_all_country_level["end_time"] = pd.to_datetime(df_all_country_level["end_time"])

df_all_country_level["Year"] = df_all_country_level["start_time"].dt.year

In [42]:
df_all_country_level.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21025 entries, 0 to 39
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   iso3_country              21025 non-null  object        
 1   sector                    21025 non-null  object        
 2   subsector                 21025 non-null  object        
 3   start_time                21025 non-null  datetime64[ns]
 4   end_time                  21025 non-null  datetime64[ns]
 5   gas                       21025 non-null  object        
 6   emissions_quantity        17465 non-null  float64       
 7   emissions_quantity_units  21025 non-null  object        
 8   temporal_granularity      15341 non-null  object        
 9   created_date              21025 non-null  object        
 10  modified_date             5413 non-null   object        
 11  Year                      21025 non-null  int32         
dtypes: datetime64[ns](2), floa

In [48]:
df_all_country_level = df_all_country_level[df_all_country_level["Year"] < 2023]

In [49]:
df_all_country_level.to_csv("df_asean.csv")

In [44]:
df_all_country_level["sector"].unique()

array(['agriculture', 'power', 'buildings', 'transportation',
       'fossil-fuel-operations', 'mineral-extraction', 'manufacturing',
       'forestry-and-land-use', 'fluorinated-gases', 'waste'],
      dtype=object)

Edit Emission on folu to use only net values

In [5]:
df = pd.read_csv("df_asean.csv")
df = df[~df["subsector"].isin(['forest-land-clearing', 'forest-land-degradation','forest-land-fires','removals', 'shrubgrass-fires', 'wetland-fires',
       'water-reservoirs'])]

df["subsector"].unique()

array(['cropland-fires', 'other-agricultural-soil-emissions',
       'manure-management-other', 'enteric-fermentation-other',
       'enteric-fermentation-cattle-feedlot',
       'manure-left-on-pasture-cattle',
       'manure-management-cattle-feedlot',
       'enteric-fermentation-cattle-pasture', 'rice-cultivation',
       'electricity-generation', 'other-energy-use',
       'residential-and-commercial-onsite-fuel-usage',
       'other-onsite-fuel-usage', 'domestic-shipping',
       'international-aviation', 'railways', 'international-shipping',
       'other-transport', 'domestic-aviation', 'road-transportation',
       'coal-mining', 'oil-and-gas-production-and-transport',
       'other-fossil-fuel-operations', 'oil-and-gas-refining',
       'solid-fuel-transformation', 'rock-quarrying', 'iron-mining',
       'sand-quarrying', 'copper-mining', 'bauxite-mining', 'cement',
       'petrochemicals', 'pulp-and-paper', 'other-manufacturing', 'steel',
       'chemicals', 'aluminum', 'net

In [6]:
df.to_csv("df_asean.csv")